# **Procesamiento de Lenguaje Natural**

## **Tarea 2 - Notebook 1**

## Integrantes

* ### Daniel Osorio Cárdenas
* ### Juan Diego Calixto Núñez

Este Notebook incluye los literales I, II, III, IV de la tarea. Estos corresponden a la construcción de los modelos de lenguaje que se usarán en el Notebook 2.

## **I. Creación de los archivos consolidados**

In [1]:
# Configurar los directorios donde se encuentran los archivos de datos
NEWS_FOLDER = '20news-18828'
BLOGS_FOLDER = 'blogs'

Empezamos cargando los archivos de 20news-18828.

In [2]:
import os

# Listar los folders de 20news-18828
news_folder = os.listdir(NEWS_FOLDER)

# Recorremos cada archivo de news
news_texts = []
for folder in news_folder:
    for file_name in os.listdir(NEWS_FOLDER + '/' + folder):
        extracted_text = ''
        with open(NEWS_FOLDER + '/' + folder + '/' + file_name, 'r', encoding='utf-8', errors='ignore') as f:
            for line in f:
                line = line.strip()
                # Agregamos solo las lineas del contenido del archivo
                if not line.startswith('From:') and not line.startswith('Subject:'):
                    extracted_text += line + " "
            news_texts.append(extracted_text + "<NEW_DOCUMENT>")

# Guardamos todos los textos en un unico .txt
with open('news_consolidated.txt', 'w', encoding='utf-8') as file:
    for text in news_texts:
        file.write(text)

# Revisamos el tamaño del archivo .txt
print('Tamaño del archivo .txt de 20news-18828: ', os.path.getsize('news_consolidated.txt'), 'bytes = ', round(os.path.getsize('news_consolidated.txt') / 1048576, 2), 'MB')

Tamaño del archivo .txt de 20news-18828:  31843073 bytes =  30.37 MB


Para cargar los archivos de blogs se hizo primero un preprocesamiento de los archivos XML puesto que hubo muchos errores a la hora de leerlos. La estrategia fue la siguiente:
* Se eliminaron los tags de formato del XML
* Se eliminaron las fechas de los posts
* Se dejó unicamente el contenido de los posts

Esto para poder leer los archivos como texto plano. Se aprovecha para saber qué palabras tienen frecuencia 1 por cada archivo para modelar el TKN \<UNK>

In [3]:
# Recorremos cada archivo de blogs
blogs_texts = []
for filename in os.listdir(BLOGS_FOLDER):
    extracted_text = ''
    with open(BLOGS_FOLDER + '/' + filename, 'r', encoding='utf-8', errors='ignore') as f:
        for line in f:
            line = line.strip()
            extracted_text += line + " "
        blogs_texts.append(extracted_text + "<NEW_DOCUMENT>")

# Guardamos todos los textos en un unico .txt
with open('blogs_consolidated.txt', 'w', encoding='utf-8') as file:
    for text in blogs_texts:
        file.write(text)

# Revisamos el tamaño del archivo .txt
print('Tamaño del archivo .txt de blogs: ', os.path.getsize('blogs_consolidated.txt'), 'bytes = ', round(os.path.getsize('blogs_consolidated.txt') / 1048576, 2), 'MB')

Tamaño del archivo .txt de blogs:  758342111 bytes =  723.21 MB


## **II y III. Crear archivos de Training y Test y Tokenizar por sentencia**

Primero se crean los archivos de training y test

In [4]:
import re

# Cargamos los textos de los archivos .txt
with open('news_consolidated.txt', 'r', encoding='utf-8', errors='ignore') as file:
    news_texts = file.read()

with open('blogs_consolidated.txt', 'r', encoding='utf-8', errors='ignore') as file:
    blogs_texts = file.read()

# Separar cada archivo en oraciones
news_texts = news_texts.split('.')
blogs_texts = blogs_texts.split('.')
print('Cantidad de oraciones en 20news-18828: ', len(news_texts))
print('Cantidad de oraciones en blogs: ', len(blogs_texts))

# Dividir los textos en 80% train y 20% test
news_texts_train = news_texts[:int(len(news_texts) * 0.8)]
news_texts_test = news_texts[int(len(news_texts) * 0.8):]
blogs_texts_train = blogs_texts[:int(len(blogs_texts) * 0.8)]
blogs_texts_test = blogs_texts[int(len(blogs_texts) * 0.8):]

# Guardar los textos de train y test en archivos .txt
with open('20N_GROUP_training.txt', 'w', encoding='utf-8') as file:
    for text in news_texts_train:
        file.write(text + '. ')

with open('20N_GROUP_testing.txt', 'w', encoding='utf-8') as file:
    for text in news_texts_test:
        file.write(text + '. ')

with open('BAC_GROUP_training.txt', 'w', encoding='utf-8') as file:
    for text in blogs_texts_train:
        file.write(text + '. ')

with open('BAC_GROUP_testing.txt', 'w', encoding='utf-8') as file:
    for text in blogs_texts_test:
        file.write(text + '. ')

# Revisamos el tamaño de los archivos .txt
print('Tamaño del archivo .txt de 20N_GROUP_training: ', os.path.getsize('20N_GROUP_training.txt'), 'bytes = ', round(os.path.getsize('20N_GROUP_training.txt') / 1048576, 2), 'MB')
print('Tamaño del archivo .txt de 20N_GROUP_testing: ', os.path.getsize('20N_GROUP_testing.txt'), 'bytes = ', round(os.path.getsize('20N_GROUP_testing.txt') / 1048576, 2), 'MB')
print('Tamaño del archivo .txt de BAC_GROUP_training: ', os.path.getsize('BAC_GROUP_training.txt'), 'bytes = ', round(os.path.getsize('BAC_GROUP_training.txt') / 1048576, 2), 'MB')
print('Tamaño del archivo .txt de BAC_GROUP_testing: ', os.path.getsize('BAC_GROUP_testing.txt'), 'bytes = ', round(os.path.getsize('BAC_GROUP_testing.txt') / 1048576, 2), 'MB')


Cantidad de oraciones en 20news-18828:  469786
Cantidad de oraciones en blogs:  13065727
Tamaño del archivo .txt de 20N_GROUP_training:  25868385 bytes =  24.67 MB
Tamaño del archivo .txt de 20N_GROUP_testing:  6444475 bytes =  6.15 MB
Tamaño del archivo .txt de BAC_GROUP_training:  625571254 bytes =  596.59 MB
Tamaño del archivo .txt de BAC_GROUP_testing:  145836585 bytes =  139.08 MB


Procedemos a cargar los archivos de entrenamiento y a tokenizar

In [9]:
# Se cargan los archivos de training
with open('20N_GROUP_training.txt', 'r', encoding='utf-8', errors='ignore') as file:
    news_texts_train = file.read()

with open('BAC_GROUP_training.txt', 'r', encoding='utf-8', errors='ignore') as file:
    blogs_texts_train = file.read()

# Normalizamos los textos pasando todo a minusculas
news_texts_train = news_texts_train.lower()
news_texts_train = news_texts_train.lower()

# Se separan los textos por "<NEW_DOCUMENT>" para obtener los documentos
news_texts_train = news_texts_train.split('<NEW_DOCUMENT>')
blogs_texts_train = blogs_texts_train.split('<NEW_DOCUMENT>')

# Para cada documento, se encuentran los términos con frecuencia 1
news_texts = ''
for text in news_texts_train:
    news_words = {}
    for word in text.split():
        if word in news_words:
            news_words[word] += 1
        else:
            news_words[word] = 1
    extracted_text = ''
    extracted_text += ' '.join([word if news_words[word] > 1 else '<UNK>' for word in text.split()])
    news_texts += extracted_text

blogs_texts = ''
for text in blogs_texts_train:
    blogs_words = {}
    for word in text.split():
        if word in blogs_words:
            blogs_words[word] += 1
        else:
            blogs_words[word] = 1
    extracted_text = ''
    extracted_text += ' '.join([word if blogs_words[word] > 1 else '<UNK>' for word in text.split()])
    blogs_texts += extracted_text

# Se reemplazan los números por el token <NUM>
news_texts= re.sub(r'\d+', '<NUM>', news_texts)
blogs_texts = re.sub(r'\d+', '<NUM>', blogs_texts)

# La marcación de TKN con frecuencia 1 se realizo en el paso anterior
# cuando se crearon los archivos consolidados

# Se separan los textos por '.' para obtener oraciones
news_texts_sentences = news_texts.split('.')
news_texts_sentences = blogs_texts.split('.')
print('Cantidad de oraciones en 20news-18828 training: ', len(news_texts))
print('Cantidad de oraciones en blogs training: ', len(blogs_texts))

Cantidad de oraciones en 20news-18828 training:  24090045
Cantidad de oraciones en blogs training:  575765517
